# Retrieval

In baseline RAG, retrieval is usually done simply by using vector search. However, as the field has developed, researchers have discovered multiple ways to enhance the retrieval process. This notebook will cover the following retrieval methods: 

- **Iterative Retrieval** - Use the reasoning capabilities of LLMs to perform iterative retrieval-generation cycles until passing an evaluation step.
- **Recursive-Retrieval** - Search for smaller documents and use chunk references or metadata references to retrieve the full document.
- **Generator-Enhanced Retrieval** - Using a LLM to predict when and what to retrieve across generation with a process called [**F**orward-**L**ooking **A**ctive **RE**trieval augmented
generation (FLARE)](https://arxiv.org/pdf/2305.06983)
- **GraphRAG** - Using existing or LLM-generated knowledge graphs to enhance retrieval by harnessing the relationships between entities.

### Setup libraries and environment

In [ ]:
%pip install ipython-ngql pyvis llama-index-readers-wikipedia llama-index-readers-papers llama-index-readers-web llama-index-graph-stores-nebula

In [ ]:
import os
import copy
import json


from IPython.display import Markdown, display

from util.helpers import (
    get_malazan_pages,
    create_and_save_md_files,
)

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    KnowledgeGraphIndex,
    PromptTemplate,
    StorageContext,
    Settings,
)
from llama_index.core.query_engine import (
    FLAREInstructQueryEngine,
    RetrieverQueryEngine,
    RetryQueryEngine,
    RetryGuidelineQueryEngine,
    KnowledgeGraphQueryEngine,
)
from llama_index.readers.papers import ArxivReader
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.evaluation import (
    RelevancyEvaluator,
    GuidelineEvaluator,
)

In [ ]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio
nest_asyncio.apply()

Add the following to a `.env` file in the root of the project if not already there.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4-turbo")
embed_model = OpenAIEmbedding(api_key=OPENAI_API_KEY, model=OpenAIEmbeddingModelType.TEXT_EMBED_3_SMALL)

Read papers from arXiv as documents

In [ ]:
reader = ArxivReader()

papers = ["2404.10981", "2305.06983", "2312.10997"]
papers_dir = "./data/docs/arxiv"

arxiv_res = [reader.load_papers_and_abstracts(search_query=f"id:{paper}", max_results=1, papers_dir=papers_dir) for paper in papers]
arxiv_documents = [doc for sublist in [d for (d, _) in arxiv_res] for doc in sublist]
abstracts = [a[0] for (_, a) in arxiv_res]
arxiv_index = VectorStoreIndex.from_documents(documents=arxiv_documents, show_progress=True)


Read articles from Malazan Wiki

In [ ]:
pages = get_malazan_pages(articles=["Anomander Rake"])
docs_path = "./data/docs/graph_rag"
create_and_save_md_files(pages, path=docs_path + "/")

malazan_documents = SimpleDirectoryReader(input_dir=docs_path).load_data()
malazan_index = VectorStoreIndex.from_documents(
    malazan_documents,
    show_progress=True,
)

Read Wiki page about Vincent Van Gogh

In [ ]:
reader = WikipediaReader()
wiki_documents = reader.load_data(pages=["Vincent Van Gogh"])
wiki_index = VectorStoreIndex.from_documents(documents=wiki_documents, show_progress=True)

## Iterative Retrieval
In **Iterative Retrieval**, the key part of the process is the evaluator or *judge* that helps self-correct the retrieval process. 

The pipeline performs the following steps:
1. first queries the base query engine, then
2. use the evaluator to decided if the response passes.
3. If the response passes, then return response,
4. Otherwise, transform the original query with the evaluation result (query, response, and feedback) into a new query,
5. Repeat up to max_retries

There's different types of evaluators that can be used, depending on the use-case. Sometimes it might be useful to evaluate the answer specifically, or the context. Other times you might need to create some guidelines for the LLM to use in order to evaluate whether to perform another retrieval step or not.

In [ ]:
## Prompt of RelevancyEvaluator
DEFAULT_EVAL_TEMPLATE = PromptTemplate(
    "Your task is to evaluate if the response for the query \
    is in line with the context information provided.\n"
    "You have two options to answer. Either YES/ NO.\n"
    "Answer - YES, if the response for the query \
    is in line with context information otherwise NO.\n"
    "Query and Response: \n {query_str}\n"
    "Context: \n {context_str}\n"
    "Answer: "
)

DEFAULT_REFINE_TEMPLATE = PromptTemplate(
    "We want to understand if the following query and response is"
    "in line with the context information: \n {query_str}\n"
    "We have provided an existing YES/NO answer: \n {existing_answer}\n"
    "We have the opportunity to refine the existing answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "If the existing answer was already YES, still answer YES. "
    "If the information is present in the new context, answer YES. "
    "Otherwise answer NO.\n"
)

In [ ]:
evaluator = RelevancyEvaluator(llm=llm)
query_engine = RetryQueryEngine(
    query_engine=arxiv_index.as_query_engine(), evaluator=evaluator
)

In [ ]:
evaluator = GuidelineEvaluator(
    llm=llm, guidelines="List elements of the answer as bullets"
)
query_engine = RetryGuidelineQueryEngine(
    query_engine=arxiv_index.as_query_engine(),
    resynthesize_query=True,
    guideline_evaluator=evaluator,
)

In [ ]:
query = "What is Advanced Retrieval Augmented Generation?"
response = query_engine.query(str_or_query_bundle=query)
display(Markdown(f"Query: {query}\n\nResponse:\n\n {response}"))


## Recursive-Retrieval

In **Recursive-Retrieval**, we search to relevant documents using references between smaller documents and their related larger documents. This enables us to optimize the narrowing the search space for the first retrieval, and then recursively increasing the context using the references.

We distinguish between two types of recursive-retrieval:
1. **Chunk-References** - References between smaller and larger documents.
2. **Metadata-References** - References between metadata like summaries or generated questions to documents.

### Chunk References

In [ ]:
node_parser = SentenceSplitter(chunk_size=1024)
base_nodes = node_parser.get_nodes_from_documents(documents=wiki_documents, show_progress=True)
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [ ]:
sub_chunk_sizes = [62, 124, 256, 512]
sub_node_parsers = [
    SentenceSplitter(chunk_size=c, chunk_overlap=20) for c in sub_chunk_sizes
]

all_nodes: list[IndexNode] = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

To test the effectiveness of chunk references go to "**Test recursive-retrieval engine**" section.

### Metadata References

In [ ]:
extractors = [
    SummaryExtractor(summaries=["self"], llm=OpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo"), show_progress=True),
    QuestionsAnsweredExtractor(questions=5, llm=OpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo"), show_progress=True),
]
# run metadata extractor across base nodes, get back dictionaries
node_to_metadata = {}
for extractor in extractors:
    metadata_dicts = extractor.extract(base_nodes)
    for node, metadata in zip(base_nodes, metadata_dicts):
        if node.node_id not in node_to_metadata:
            node_to_metadata[node.node_id] = metadata
        else:
            node_to_metadata[node.node_id].update(metadata)

In [ ]:
def save_metadata_dicts(path, data):
    with open(path, "w") as fp:
        json.dump(data, fp)


def load_metadata_dicts(path):
    with open(path, "r") as fp:
        data = json.load(fp)
    return data


In [ ]:

path = "./data/retrieval"
if not os.path.exists(path):
    print("Creating directory: ", path)
    os.makedirs(path)

save_metadata_dicts(path + "/metadata_dicts.json", node_to_metadata)


In [ ]:
metadata_dicts = load_metadata_dicts("./data/retrieval/metadata_dicts.json")

In [ ]:
all_nodes = copy.deepcopy(base_nodes)
for node_id, metadata in node_to_metadata.items():
    for val in metadata.values():
        all_nodes.append(IndexNode(text=val, index_id=node_id))


#### Test recursive-retrieval engine

In [ ]:

all_nodes_dict = {n.node_id: n for n in all_nodes}
recursive_index = VectorStoreIndex(all_nodes, embed_model=embed_model, show_progress=True)


In [ ]:
retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": recursive_index.as_retriever()},
    node_dict=all_nodes_dict,
    verbose=True,
)
recursive_query_engine = RetrieverQueryEngine.from_args(retriever=retriever, llm=llm)

In [ ]:
nodes = retriever.retrieve("Where did Vincent Van Gogh live during his life?")

for node in nodes:
    display(Markdown(f"### {node.node_id}"))
    display(Markdown(f"{node.text}"))

In [ ]:
response = recursive_query_engine.query("Where did Vincent Van Gogh live during his life?")

display(Markdown(f'{response}'))

## Generator-Enhanced Retrieval

The following example will demonstrate the **F**orward **L**ooking **A**ctive **RE**trieval augmented generation (FLARE). This method actively uses the LLM to help with retrieval by generating retrieval instructions. FLARE has the following methodology:

- **Multiple retrievals**: FLARE uses multiple retrievals based on queries created by the LLM.
- **When to retrieve**: Uses LLM to decide when to retrieve.
- **What to retrieve**: Uses LLM to decide what to retrieve.
- **Iterative**: Iteratively expands the answer by "looking ahead" to decide what to retrieve next.

The following is the instruct prompt used in each iteration:

```
Skill 1. Use the Search API to look up relevant information by writing [Search(query)]" where "query" is the search query you want to look up.
For example:

Query: But what are the risks during production of nanomaterials?
Answer: [Search(What are some nanomaterial production risks?)]

Query: The colors on the flag of Ghana have the following meanings.
Answer: Red is for [Search(What is the meaning of Ghana's flag being red?)], green for forests, and gold for mineral wealth.

Query: What did the author do during his time in college?
Answer: The author took classes in [Search(What classes did the author take in college?)].
------------------------------------------------------------
Skill 2. Solve more complex generation tasks by thinking step by step. For example:

Query: Give a summary of the author's life and career.
Answer: The author was born in 1990. Growing up, he [Search(What did the author do during his childhood?)].

Query: Can you write a summary of the Great Gatsby.
Answer: The Great Gatsby is a novel written by F. Scott Fitzgerald. It is about [Search(What is the Great Gatsby about?)].
------------------------------------------------------------
Now given the following task, and the stub of an existing answer, generate the next portion of the answer. 
You may use the Search API "[Search(query)]" whenever possible.
If the answer is complete and no longer contains any "[Search(query)]" tags, write "done" to finish the task.
Do not write "done" if the answer still contains "[Search(query)]" tags.
Do not make up answers. It is better to generate one "[Search(query)]" tag and stop generation than to fill in the answer with made up information with no "[Search(query)]" tags or multiple "[Search(query)]" tags that assume a structure in the answer.
Try to limit generation to one sentence if possible.


Query: {query_str}
Existing Answer: {existing_answer}
Answer: 
```

The prompt uses *Few-shot* prompting to define three skills:
- **Search API skill**: Gives the LLM the ability to create *query tasks* for information in the next iteration.
- **Step-by-step generation skill**: Gives the LLM the ability answer more complex questions by thinking step by step.
- **"When to stop" skill**: Check when the answer is complete by looking for uses of the "*Search API*".

After each iteration all query tasks are parsed and the queries are used to retrieve information. The retrieved information is then parsed and then used to "Look ahead" to what the next query could be. The following is the *lookahead* prompt:

```
An existing 'lookahead response' is given below. The lookahead response contains `[Search(query)]` tags. Some queries have been executed and the response retrieved. The queries and answers are also given below.
Also the previous response (the response before the lookahead response) is given below.
Given the lookahead template, previous response, and also queries and answers, please 'fill in' the lookahead template with the appropriate answers.

NOTE: Please make sure that the final response grammatically follows the previous response + lookahead template. For example, if the previous response is "New York City has a population of " and the lookahead template is "[Search(What is the population of New York City?)]", then the final response should be "8.4 million".

NOTE: the lookahead template may not be a complete sentence and may contain trailing/leading commas, etc. Please preserve the original formatting of the lookahead template if possible.

NOTE: the exception to the above rule is if the answer to a query is equivalent to "I don't know" or "I don't have an answer". In this case, modify the lookahead template to indicate that the answer is not known.

NOTE: the lookahead template may contain multiple `[Search(query)]` tags and only a subset of these queries have been executed. Do not replace the `[Search(query)]` tags that have not been executed.
------------------------------------------------------------
Previous Response:

Lookahead Template: Red is for [Search(What is the meaning of Ghana's flag being red?)], green for forests, and gold for mineral wealth.

Query-Answer Pairs:
Query: What is the meaning of Ghana's flag being red?
Answer: The red represents the blood of those who died in the country's struggle \
    for independence

Filled in Answers: Red is for the blood of those who died in the country's struggle for independence, green for forests, and gold for mineral wealth.
------------------------------------------------------------
Previous Response: One of the largest cities in the world

Lookahead Template: , the city contains a population of [Search(What is the population of New York City?)]

Query-Answer Pairs:
Query: What is the population of New York City?
Answer: The population of New York City is 8.4 million

Synthesized Response: , the city contains a population of 8.4 million
------------------------------------------------------------
Previous Response: the city contains a population of

Lookahead Template: [Search(What is the population of New York City?)]

Query-Answer Pairs:
Query: What is the population of New York City?
Answer: The population of New York City is 8.4 million

Synthesized Response: 8.4 million
------------------------------------------------------------
Previous Response:
{prev_response}

Lookahead Template:
{lookahead_response}

Query-Answer Pairs:
{query_answer_pairs}

Synthesized Response:
```

In [ ]:
flare_query_engine = FLAREInstructQueryEngine(
    llm=llm,
    query_engine=malazan_index.as_query_engine(), 
    max_iterations=5, 
    verbose=True
)

In [ ]:
response = flare_query_engine.query("Tell me about Anomander Rake")

display(Markdown(f"<b>{response}</b>"))

## GraphRAG

In this section we will have a look at an example of using knowledge graphs to enhance retrieval. We will show how you can use LLMs to generate our knowledge graphs for us based on arbitrary documents and then use these to search for relevant contexts to supply to the LLM.

If you don't already have NebulaGraph running locally, you can use [nebula-up](https://github.com/wey-gu/nebula-up) to start it up.

Run the following command in your terminal (On Windows you should use WSL):

```bash
curl -fsSL nebula-up.siwei.io/install.sh | bash
```



If your NebulaGraph is started with nebula-up, you can add the following to your .env file:

```
NEBULA_USER=root
NEBULA_PASSWORD=nebula
NEBULA_HOST=localhost
NEBULA_PORT=9669
```

Or set with your own configurations for NebulaGraph.

In [ ]:
%load_ext ngql


In [ ]:
load_dotenv(override=True)
GRAPHD_HOST = os.getenv("NEBULA_HOST")
GRAPHD_PORT = os.getenv("NEBULA_PORT")
NEBULA_PASSWORD = os.getenv("NEBULA_PASSWORD")
NEBULA_USER = os.getenv("NEBULA_USER")
NEBULA_ADDRESS = f"{GRAPHD_HOST}:{GRAPHD_PORT}"
os.environ["NEBULA_ADDRESS"] = NEBULA_ADDRESS

In [ ]:
connection_string = f"--address {GRAPHD_HOST} --port {GRAPHD_PORT} --user {NEBULA_USER} --password {NEBULA_PASSWORD}"
%ngql {connection_string}

Create new knowledge graph (**Graph Space** in Nebula terms) to use for retrieval. We'll create an entity-relationship graph with the following schema:
```
[entity:tag] - [relationship:edge] -> [entity:tag]
```

This will allow the LLM to automatically generate our knowledge graph with arbitrary relationships like:
```
[entity:David] - [relationship:has studied computer science at] -> [entity:Aarhus University]
[entity:David] - [relationship:currently works as a software pilot at] -> [entity:Trifork A/S]
```

We also generate an index for the name

In [ ]:
%ngql CREATE SPACE IF NOT EXISTS graph_rag(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
%ngql SHOW SPACES

In [ ]:
%%ngql
USE graph_rag;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);
CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

In [ ]:
#%ngql USE graph_rag; CLEAR SPACE graph_rag; # clean graph space

Now we can use LlamaIndex's `KnowledgeGraphIndex` to index the graph and use it for retrieval. It uses the following default prompt to generate triplets for the graph:

   ```
    Some text is provided below. Given the text, extract up to 
    {max_knowledge_triplets} 
    knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.
    ---------------------
    Example:
    Text: Alice is Bob's mother.
    Triplets:
    (Alice, is mother of, Bob)
    Text: Philz is a coffee shop founded in Berkeley in 1982.
    Triplets:
    (Philz, is, coffee shop)
    (Philz, founded in, Berkeley)
    (Philz, founded in, 1982)
    ---------------------
    Text: {text}
    Triplets:
```

In [ ]:
os.environ["NEBULA_ADDRESS"] = NEBULA_ADDRESS
os.environ["NEBULA_PASSWORD"] = NEBULA_PASSWORD
os.environ["NEBULA_USER"] = NEBULA_USER

space_name = "graph_rag"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

Settings.llm = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4-turbo")
Settings.embed_model = OpenAIEmbedding(api_key=OPENAI_API_KEY, model=OpenAIEmbeddingModelType.TEXT_EMBED_3_SMALL)
Settings.chunk_size = 512
storage_context = StorageContext.from_defaults(graph_store=graph_store)

This may take a little while. Afterwards we save the index to disk so we don't have to do it again.

In [ ]:
kg_index = KnowledgeGraphIndex.from_documents(
    malazan_documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    show_progress=True,
)

In [ ]:
# kg_index.storage_context.persist(persist_dir='./data/storage_graph')

In [ ]:
# from llama_index.core import load_index_from_storage

# storage_context = StorageContext.from_defaults(
#     persist_dir="./data/storage_graph", graph_store=graph_store
# )
# kg_index = load_index_from_storage(
#     storage_context=storage_context,
#     max_triplets_per_chunk=10,
#     space_name=space_name,
#     edge_types=edge_types,
#     rel_prop_names=rel_prop_names,
#     tags=tags,
#     verbose=True,
# )

In [ ]:
response = kg_index.as_retriever().retrieve("Anomander Rakes friendship with Caladan Brood")
display(Markdown(f'{"\n\nAnomander{name: Anomander}".join(response[-1].text.split("Anomander{name: Anomander}")[:10])}'))

In [ ]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [ ]:
%ngql MATCH p=(n)-[e:relationship*1..2]-(m) WHERE id(n) in ['Anomander'] RETURN n.entity.name,e[0].relationship, m.entity.name LIMIT 10

In [ ]:
response = kg_index_query_engine.query("Tell me about Anomander Rakes relationship with the Malazan Empire")

In [ ]:
display(Markdown(f'{response}'))

### NL2Cypher

It's also possible that we might already have existing knowledge graphs with useful information which hasn't been indexed with embeddings. This would prohibit us from using the vector index to find the right entities/relationships. In this case its possible to use LLMs to automatically generate Cypher queries to retrieve information from the graph based on the query.

NL2Cypher is a tool that can be used to generate Cypher queries from natural language queries. We can use it to generate Cypher queries for our knowledge graph.

Currently this tool doesn't work super well with the Knowledge Space generated about "Anomander Rake", this shows how you might use it with a different knowledge graph.
Some fine-tuning might be needed to get the better results.


In [ ]:
DEFAULT_NEBULAGRAPH_NL2CYPHER_PROMPT_TMPL = """
Generate NebulaGraph query from natural language.
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
---
{schema}
---
Note: NebulaGraph speaks a dialect of Cypher, comparing to standard Cypher:

1. it uses double equals sign for comparison: `==` rather than `=`
2. it needs explicit label specification when referring to node properties, i.e.
v is a variable of a node, and we know its label is Foo, v.`foo`.name is correct
while v.name is not.

For example, see this diff between standard and NebulaGraph Cypher dialect:
```diff
< MATCH (p:person)-[:directed]->(m:movie) WHERE m.name = 'The Godfather'
< RETURN p.name;
---
> MATCH (p:`person`)-[:directed]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather'
> RETURN p.`person`.`name`;
```

Question: {query_str}

NebulaGraph Cypher dialect query:
"""
DEFAULT_NEBULAGRAPH_NL2CYPHER_PROMPT = PromptTemplate(
    DEFAULT_NEBULAGRAPH_NL2CYPHER_PROMPT_TMPL,
    prompt_type=PromptType.TEXT_TO_GRAPH_QUERY,
)

query_engine_with_nl2graphquery =  KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    llm=OpenAI(api_key=OPENAI_API_KEY),
    verbose=True,
    graph_query_synthesis_prompt=DEFAULT_NEBULAGRAPH_NL2CYPHER_PROMPT
)

In [ ]:
query_engine_with_nl2graphquery.generate_query("Tell me Anomander's relationship with Andarist")

In [ ]:
response = query_engine_with_nl2graphquery.query(
    "Tell me about Anomander's relationship Andarist",
)
display(Markdown(f"<b>{response}</b>"))